In [3]:
# reflected location error in ozone data simulation

import torch
import torch.fft
import numpy as np
import sys
import os
import logging
import argparse 
import pandas as pd
import pickle
import torch.optim as optim
import copy
import time
from sklearn.neighbors import BallTree
from typing import Optional, List, Tuple
from pathlib import Path

# --- CUSTOM PATHS ---
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

# (필요 시 실제 GEMS_TCO 라이브러리 import)
try:
    from GEMS_TCO import kernels_for_simulation_020626 as kernels_simulation
    
    # from GEMS_TCO import kernels_reparam_space_time_gpu_copy as kernels_reparam
    
    from GEMS_TCO import orderings as _orderings
    from GEMS_TCO import alg_optimization, BaseLogger


except ImportError:
    print("Warning: GEMS_TCO modules not found. Ensure paths are correct.")

from GEMS_TCO import configuration as config
from GEMS_TCO.data_loader import load_data2, exact_location_filter
from GEMS_TCO import debiased_whittle





In [4]:
space: List[str] = ['1', '1']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 8
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)

#lat_range_input = [1, 3]
#lon_range_input = [125.0, 129.0]

lat_range_input=[-3,-1]      
lon_range_input=[121, 125] 

#lat_range_input=[-3,2]      
#lon_range_input=[121, 131] 

# 기존: df_map, ord_mm, nns_map, day_offsets = ...
# 수정 후: 변수명을 monthly_mean으로 변경

df_map, ord_mm, nns_map, monthly_mean = data_load_instance.load_maxmin_ordered_data_bymonthyear(
    lat_lon_resolution=lat_lon_resolution, 
    mm_cond_number=mm_cond_number,
    years_=years, 
    months_=month_range,
    lat_range=lat_range_input,   
    lon_range=lon_range_input
)

print(f"Global Monthly Mean: {monthly_mean}") # 확인용 출력


daily_aggregated_reg_vecc = [] 
daily_hourly_maps_reg_vecc = []      

daily_aggregated_irr_vecc = [] 
daily_hourly_maps_irr_vecc = []   


for day_index in range(31):
    hour_start_index = day_index * 8
    
    hour_end_index = (day_index + 1) * 8
    #hour_end_index = day_index*8 + 1
    hour_indices = [hour_start_index, hour_end_index]

    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
    df_map, 
    monthly_mean,  # <--- 이 부분이 추가되어야 합니다
    hour_indices, 
    ord_mm=ord_mm,
    dtype=torch.float64, 
    keep_ori=False
    )

    daily_aggregated_reg_vecc.append( day_aggregated_tensor )
    daily_hourly_maps_reg_vecc.append( day_hourly_map )

    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
    df_map, 
    monthly_mean,  # <--- 이 부분이 추가되어야 합니다
    hour_indices, 
    ord_mm=ord_mm,
    dtype=torch.float64, 
    keep_ori= True
    )

    daily_aggregated_irr_vecc.append( day_aggregated_tensor )
    daily_hourly_maps_irr_vecc.append( day_hourly_map )
print(daily_aggregated_irr_vecc[0].shape)

nn = daily_aggregated_irr_vecc[0].shape[0]

--- Global Monthly Mean for 2024-7: 257.0667 ---
Global Monthly Mean: 257.0666873781436
torch.Size([22680, 11])


In [5]:
import torch
import torch.nn.functional as F
import numpy as np
from sklearn.neighbors import BallTree
import sys

# --- [설정] 오존 농도 시뮬레이션 파라미터 ---
# 1. U-Shape Diurnal Cycle (8시간: 08:00 ~ 15:00 가정)
# 예: 아침에 높았다가(10), 낮에 떨어지고(-5), 오후에 다시 오르는 패턴
# 실제 데이터의 beta 계수와 비슷한 스케일로 설정하세요.
HOURLY_TREND = torch.tensor([10.0, 5.0, 0.0, -5.0, -8.0, -5.0, 0.0, 5.0], dtype=torch.float64)

# 2. Daily Baseline (월 평균)
MONTHLY_MEAN = 260.0

# --- 2. EXACT COVARIANCE & FFT HELPERS (기존 유지) ---
def make_target_grid(lat_start, lat_end, lat_step, lon_start, lon_end, lon_step, device, dtype):
    if lat_start > lat_end and lat_step > 0: lat_step = -lat_step
    if lon_start > lon_end and lon_step > 0: lon_step = -lon_step
    lats = torch.arange(lat_start, lat_end - 0.0001, lat_step, device=device, dtype=dtype)
    lons = torch.arange(lon_start, lon_end + 0.0001, lon_step, device=device, dtype=dtype)
    lats = torch.round(lats * 10000) / 10000
    lons = torch.round(lons * 10000) / 10000
    grid_lat, grid_lon = torch.meshgrid(lats, lons, indexing='ij')
    center_points = torch.stack([grid_lat.flatten(), grid_lon.flatten()], dim=1)
    return center_points, len(lats), len(lons)

# --- 2. High-Res Field Generator (기존 유지) ---
def generate_high_res_field(target_lat_range, target_lon_range, t_steps, params, device, dtype):
    # ... (FFT 로직 기존과 동일하므로 생략, 위 코드 그대로 사용) ...
    # [설정] 해상도 확대 배율
    lat_res_factor = 160.0 
    lon_res_factor = 4.0   
    lat_res_high = 0.044 / lat_res_factor
    lon_res_high = 0.063 / lon_res_factor
    
    t_lat_max = max(target_lat_range)
    t_lat_min = min(target_lat_range)
    
    lats_high = torch.arange(t_lat_max + 0.1, t_lat_min - 0.1, -lat_res_high, device=device, dtype=dtype)
    lons_high = torch.arange(target_lon_range[0] - 0.1, target_lon_range[1] + 0.1, lon_res_high, device=device, dtype=dtype)
    
    Nx, Ny, Nt = len(lats_high), len(lons_high), t_steps
    dlat, dlon, dt = lat_res_high, lon_res_high, 1.0
    
    Px, Py, Pt = 2*Nx, 2*Ny, 2*Nt
    lags_x = torch.arange(Px, device=device, dtype=dtype) * dlat; lags_x[Px//2:] -= (Px * dlat)
    lags_y = torch.arange(Py, device=device, dtype=dtype) * dlon; lags_y[Py//2:] -= (Py * dlon)
    lags_t = torch.arange(Pt, device=device, dtype=dtype) * dt;   lags_t[Pt//2:] -= (Pt * dt)

    L_x, L_y, L_t = torch.meshgrid(lags_x, lags_y, lags_t, indexing='ij')
    
    phi1, phi2 = torch.exp(params[0]), torch.exp(params[1])
    phi3, phi4 = torch.exp(params[2]), torch.exp(params[3])
    adv_lat, adv_lon = params[4], params[5]
    
    sigma_sq = phi1 / phi2 
    
    u_x = L_x - adv_lat * L_t
    u_y = L_y - adv_lon * L_t
    dist_sq = (u_x * torch.sqrt(phi3) * phi2)**2 + (u_y * phi2)**2 + (L_t * torch.sqrt(phi4) * phi2)**2
    C_vals = sigma_sq * torch.exp(-torch.sqrt(dist_sq + 1e-12))

    S = torch.fft.fftn(C_vals); S.real = torch.clamp(S.real, min=0)
    random_phase = torch.fft.fftn(torch.randn(Px, Py, Pt, device=device, dtype=dtype))
    field_sim_raw = torch.fft.ifftn(torch.sqrt(S.real) * random_phase).real
    
    field_sim = field_sim_raw[:Nx, :Ny, :Nt]
    current_std = field_sim.std()
    target_std = torch.sqrt(sigma_sq)
    field_calibrated = (field_sim - field_sim.mean()) * (target_std / (current_std + 1e-9))
    
    return field_calibrated, lats_high, lons_high

# --- 3. Main Generator (Vecc & DW) - [핵심 수정 부분] ---
def generate_exact_count_datasets_fixed(daily_maps_real, true_params_tensor, target_grid_info, device, dtype):
    """
    Step 1 (Daily AR1) + Step 2 (Hourly Trend) + Step 3 (GP)를 모두 결합합니다.
    """
    lat_s, lat_e, lat_step, lon_s, lon_e, lon_step = target_grid_info
    
    # [A] 타겟 격자 및 High-Res Field 준비
    target_grid_coords, Nx, Ny = make_target_grid(lat_s, lat_e, lat_step, lon_s, lon_e, lon_step, device, dtype)
    target_tree = BallTree(np.radians(target_grid_coords.cpu().numpy()), metric='haversine') 
    
    # 8시간치 GP 생성
    high_res_field, lats_high, lons_high = generate_high_res_field((lat_s, lat_e), (lon_s, lon_e), 8, true_params_tensor, device, dtype)
    
    hr_mesh_lat, hr_mesh_lon = torch.meshgrid(lats_high, lons_high, indexing='ij')
    hr_tree = BallTree(np.radians(torch.stack([hr_mesh_lat.flatten(), hr_mesh_lon.flatten()], dim=1).cpu().numpy()), metric='haversine')
    high_res_flat = high_res_field.reshape(-1, 8) 

    irr_map_dict, dw_map_dict = {}, {}
    irr_list, dw_list = [], []
    noise_std = torch.sqrt(torch.exp(true_params_tensor[6]))

    # [Step 1] Daily Baseline 생성 (AR1 Process 가정)
    # 여기서는 하루치만 생성하므로, AR(1)의 특정 시점 값 하나를 랜덤 생성하거나 고정합니다.
    # 만약 여러 날짜를 루프 돈다면 e_d = phi * e_{d-1} + noise 로 갱신해야 합니다.
    # 예시: 오늘 날씨에 따른 편차 (예: +5.0)
    daily_weather_noise = torch.randn(1, device=device, dtype=dtype) * 2.0 # AR process의 sigma
    daily_base_val = MONTHLY_MEAN + daily_weather_noise # 오늘의 기저 농도
    
    print(f"   [Simulation Info] Daily Base: {daily_base_val.item():.2f} (MonthMean {MONTHLY_MEAN} + Noise {daily_weather_noise.item():.2f})")

    # [C] 데이터 매핑
    day0_dict = daily_maps_real[0]
    sorted_keys = sorted([k for k in day0_dict.keys() if 'hm' in k or 'time' in k])
    
    # GPU로 미리 Trend 이동
    hourly_trend_gpu = HOURLY_TREND.to(device)

    for t_idx, key in enumerate(sorted_keys):
        if t_idx >= 8: break
        real_tensor = day0_dict[key].to(device)
        
        clean_tensor = real_tensor 
        real_locs = clean_tensor[:, :2]
        N_points = len(real_locs)
        
        # 1. High-Res GP 값 샘플링 (Step 3)
        _, hr_indices = hr_tree.query(np.radians(real_locs.cpu().numpy()), k=1)
        gp_signal = high_res_flat[torch.tensor(hr_indices.flatten(), device=device), t_idx]
        
        # 2. Hourly Trend 가져오기 (Step 2)
        current_hour_trend = hourly_trend_gpu[t_idx]
        
        # 3. [핵심] 시뮬레이션 값 합성
        # Value = (Daily Base) + (Hourly Trend) + (GP Residual) + (Nugget Noise)
        sim_vals = daily_base_val + current_hour_trend + gp_signal + (torch.randn_like(gp_signal) * noise_std)
        
        # 4. Time Dummies 생성 (Feature)
        dummies = F.one_hot(torch.tensor([t_idx]), num_classes=8).repeat(N_points, 1)
        dummies = dummies[:, 1:].to(device=device, dtype=dtype) # D1 ~ D7 (Intercept가 D0 역할 혹은 별도)
        
        # 5. Offset Column 생성 (Step 1만 포함!)
        # 모델에게 "Hourly Trend"는 숨기고(Time Dummy로 맞추게 하고), "Daily Base"는 알려줌(Offset으로 뺌)
        # 만약 모델이 Intercept도 학습하게 하려면 offset을 0으로 두거나 MONTHLY_MEAN만 줘도 됨.
        # 여기서는 "Step 1(일별 변동)은 물리 모델이 잡아줬다"고 가정하고 Offset에 넣음.
        offset_col = torch.full((N_points, 1), daily_base_val.item(), device=device, dtype=dtype)
        
        # --- 결합 ---
        # (1) Vecc
        base_vecc = torch.stack([real_locs[:,0], real_locs[:,1], sim_vals, clean_tensor[:,3]], dim=1)
        vecc_row = torch.cat([base_vecc, dummies, offset_col], dim=1).detach() 
        irr_map_dict[key] = vecc_row
        irr_list.append(vecc_row)
        
        # (2) DW
        current_grid_points = target_grid_coords[:N_points]
        base_dw = torch.stack([current_grid_points[:,0], current_grid_points[:,1], sim_vals, clean_tensor[:,3]], dim=1)
        dw_row = torch.cat([base_dw, dummies, offset_col], dim=1).detach() 
        dw_map_dict[key] = dw_row
        dw_list.append(dw_row)

    return [torch.cat(dw_list)], [dw_map_dict], [torch.cat(irr_list)], [irr_map_dict]

In [8]:
# --- 4. EXECUTION ---
import numpy as np
import torch

# 1. 환경 설정
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DTYPE = torch.float64

# 타겟 그리드 정보 (Lat Start, End, Step, Lon Start, End, Step)
target_grid_info = (-1.0, -3.0, 0.044, 121.0, 125.0, 0.063)

# 2. True Parameter 정의 (정답지)
# 이 파라미터로 High-Res GP를 생성합니다.
init_sigmasq    = 13.059
init_range_lon  = 0.195 
init_range_lat  = 0.154 
init_range_time = 1.0
init_nugget     = 0.247
init_advec_lat  = 0.0418
init_advec_lon  = -0.1689

# 커널 내부 파라미터(Phi)로 변환
true_phi2 = 1.0 / init_range_lon              
true_phi1 = init_sigmasq * true_phi2          
true_phi3 = (init_range_lon / init_range_lat)**2
true_phi4 = (init_range_lon / init_range_time)**2

# 텐서 형태로 변환 (Log Scale for positive constraints)
true_params_tensor = [
    torch.tensor([np.log(true_phi1)], device=DEVICE, dtype=DTYPE), # Log Phi1
    torch.tensor([np.log(true_phi2)], device=DEVICE, dtype=DTYPE), # Log Phi2
    torch.tensor([np.log(true_phi3)], device=DEVICE, dtype=DTYPE), # Log Phi3
    torch.tensor([np.log(true_phi4)], device=DEVICE, dtype=DTYPE), # Log Phi4
    torch.tensor([init_advec_lat],    device=DEVICE, dtype=DTYPE), # Advec Lat
    torch.tensor([init_advec_lon],    device=DEVICE, dtype=DTYPE), # Advec Lon
    torch.tensor([np.log(init_nugget)], device=DEVICE, dtype=DTYPE)# Log Nugget
]

print(f"--- Simulation Configuration ---")
print(f"Target Sigma^2: {init_sigmasq}")
print(f"Hourly Trend applied: {HOURLY_TREND.cpu().numpy()}")
print(f"Daily Base applied: AR(1) logic inside function")

# 3. 데이터 생성 함수 호출
# 주의: 'daily_hourly_maps_vecc'는 이전 셀(데이터 로드 부분)에서 이미 로드되어 있어야 합니다.
if 'daily_hourly_maps_irr_vecc' in locals() and len(daily_hourly_maps_irr_vecc) > 0:
    print("\n🚀 Generating Simulated Data (Step 1+2+3)...")
    
    (daily_aggregated_reg_vecc_sim, daily_hourly_maps_reg_vecc_sim,
     daily_aggregated_irr_vecc_sim, daily_hourly_maps_irr_vecc_sim) = generate_exact_count_datasets_fixed(
        daily_hourly_maps_irr_vecc,  # 원본 데이터(위치 정보용)
        true_params_tensor,      # GP 파라미터
        target_grid_info,        # 그리드 정보
        DEVICE, 
        DTYPE
    )
    
    # 4. 결과 검증
    sim_tensor = daily_aggregated_irr_vecc_sim[0] # 첫째 날 데이터
    
    print(f"\n✅ Simulation Complete!")
    print(f"   - Generated Tensor Shape: {sim_tensor.shape}")
    print(f"     (Cols: Lat, Lon, Val, Time, D1..D7, Offset)")
    
    # 값 확인 (Simulated Val - Offset)이 GP+Trend 잔차와 비슷한지
    # Val(col 2) - Offset(col 11)
    # Col 11은 인덱스 11 (12번째 컬럼)
    if sim_tensor.shape[1] >= 12:
        residuals = sim_tensor[:, 2] - sim_tensor[:, 11]
        print(f"   - Residual Stats (Val - Offset):")
        print(f"     Mean: {residuals.mean().item():.4f} (Should be close to Mean of Hourly Trend)")
        print(f"     Std : {residuals.std().item():.4f}  (Should contain Sigma_sq + Trend Var)")
    else:
        print("   ⚠️ Warning: Column count is less than 12. Check dummy/offset creation.")

else:
    print("\n❌ Error: 'daily_hourly_maps_vecc' variable not found.")
    print("   Please run the Data Loading cell (Cell 3 or 7 in your notebook) first.")

--- Simulation Configuration ---
Target Sigma^2: 13.059
Hourly Trend applied: [10.  5.  0. -5. -8. -5.  0.  5.]
Daily Base applied: AR(1) logic inside function

🚀 Generating Simulated Data (Step 1+2+3)...
   [Simulation Info] Daily Base: 261.75 (MonthMean 260.0 + Noise 1.75)

✅ Simulation Complete!
   - Generated Tensor Shape: torch.Size([22680, 12])
     (Cols: Lat, Lon, Val, Time, D1..D7, Offset)
   - Residual Stats (Val - Offset):
     Mean: 0.2174 (Should be close to Mean of Hourly Trend)
     Std : 7.0562  (Should contain Sigma_sq + Trend Var)


# Fit vecchia max min time 2 

In [22]:
import torch
import numpy as np
import time

# --- CONFIGURATION ---
v = 0.5              # Smoothness
mm_cond_number = 8   # Neighbors
#mm_cond_number = 16   # Neighbors
nheads = 300           # 0 = Pure Vecchia
lr = 1.0             # LBFGS learning rate
LBFGS_MAX_STEPS = 3
LBFGS_HISTORY_SIZE = 100 # 100
LBFGS_LR = 1.0
LBFGS_MAX_EVAL = 30    

#DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- 1. SETUP PARAMETERS (List of Scalars) ---
# Truth: [4.18, 1.94, 0.24, -3.97, 0.014, -0.20, -0.85]
init_sigmasq   = 13.059
init_range_lat = 0.154 
init_range_lon = 0.195
init_advec_lat = 0.0218
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247

# Map model parameters to the 'phi' reparameterization
init_phi2 = 1.0 / init_range_lon                # 1/range_lon
init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

# Create Initial Parameters (Float64, Requires Grad)
initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]

# [4.2042, 1.6348, 0.4721, -3.2695, 0.0218, -0.1689, -1.3984]
params_list = [
    torch.tensor([val], requires_grad=True, dtype=torch.float64, device=DEVICE)
    for val in initial_vals
]

# --- 2. INSTANTIATE MODEL ---
print(f'\n{"="*40}')
print(f'--- Initializing VecchiaBatched Model ---')
print(f'{"="*40}')

if isinstance(daily_aggregated_irr_vecc_sim, torch.Tensor):
    daily_aggregated_irr_vecc_sim = daily_aggregated_irr_vecc_sim.to(DEVICE)

# Instantiate
model_instance = kernels_ar.fit_vecchia_lbfgs(
    smooth=v,
    #input_map=daily_hourly_maps_vecc_sim[0],
    #aggregated_data= daily_aggregated_tensors_vecc_sim[0],

    input_map=daily_hourly_maps_reg_vecc_sim[0],
    aggregated_data= daily_aggregated_reg_vecc_sim[0],
    nns_map=nns_map,
    mm_cond_number=mm_cond_number,
    nheads=nheads
)

'''
model_instance = kernels_reparam_space_time_gpu_col.fit_vecchia_lbfgs(
    smooth=v,
    #input_map=daily_hourly_maps_vecc_sim[0],
    #aggregated_data= daily_aggregated_tensors_vecc_sim[0],

    input_map=daily_hourly_maps_irr_vecc_sim[0],
    aggregated_data= daily_aggregated_irr_vecc_sim[0],

    nns_map=None,
    mm_cond_number=mm_cond_number
)
''' 

# --- 3. OPTIMIZATION LOOP ---
print(f'\n{"="*40}')
print(f'--- Running L-BFGS Optimization ---')
print(f'{"="*40}')

# Optimizer takes the LIST of scalars
optimizer_vecc = model_instance.set_optimizer(
            params_list,     
            lr=LBFGS_LR,            
            max_iter=LBFGS_MAX_EVAL,        
            history_size=LBFGS_HISTORY_SIZE 
        )

start_time = time.time()

out, steps_ran = model_instance.fit_vecc_lbfgs(
        params_list,
        optimizer_vecc,
        # covariance_function argument is GONE
        max_steps=LBFGS_MAX_STEPS, 
        grad_tol=1e-7
    )


end_time = time.time()
epoch_time = end_time - start_time

print(f"\nOptimization finished in {epoch_time:.2f}s.")
print(f"Results after {steps_ran} steps: {out}")
print(f"Final Params: {torch.cat(params_list).detach().cpu().numpy()}")

Using device: cpu

--- Initializing VecchiaBatched Model ---

--- Running L-BFGS Optimization ---
🚀 Pre-computing (Time Dummies & Latitude Centering)... [Mean Lat: -1.9526] ✅ Done. (Heads: 2400, Tails: 20280)
--- Starting Batched L-BFGS Optimization (GPU) ---
--- Step 1/3 / Loss: 1.806781 ---
  Param 0: Value=4.9324, Grad=-0.0012334454438898586
  Param 1: Value=2.5006, Grad=0.001210990819328973
  Param 2: Value=0.2170, Grad=-0.0008502358196047664
  Param 3: Value=-4.9371, Grad=-0.0013332741630110228
  Param 4: Value=0.0975, Grad=-0.002917004343360237
  Param 5: Value=-0.0322, Grad=0.0004220682113979697
  Param 6: Value=1.2546, Grad=-0.0016858415066192982
  Max Abs Grad: 2.917004e-03
------------------------------
--- Step 2/3 / Loss: 1.765995 ---
  Param 0: Value=-7.7786, Grad=0.00015935633782689912
  Param 1: Value=-13.1935, Grad=-0.00015932134294813235
  Param 2: Value=-26.5020, Grad=2.4468541666580601e-26
  Param 3: Value=29.3370, Grad=0.000259285064588584
  Param 4: Value=1.0990, G

In [ ]:
irr

Final Interpretable Params: {'sigma_sq': 13.190530313067597, 'range_lon': 0.19707111772283428, 'range_lat': 0.1527527611913852, 'range_time': 0.9929672518527729, 'advec_lat': -0.02957260581397776, 'advec_lon': -0.16218858835449518, 'nugget': 0.23616033677518194}

Optimization finished in 12.21s.
Results after 2 steps: [4.203689783333332, 1.6241906117210854, 0.5094879856667579, -3.234266034480146, -0.02957260581397776, -0.16218858835449518, -1.4432443115243747, 1.1447534519147522]
Final Params: [ 4.20368978  1.62419061  0.50948799 -3.23426603 -0.02957261 -0.16218859
 -1.44324431]

reg

dw

Final Interpretable Params: {'sigma_sq': 13.048806443143716, 'range_lon': 0.2759775584138689, 'range_lat': 0.26948439045514627, 'range_time': 1.2123116662337672, 'advec_lat': 0.003713422682726235, 'advec_lon': -0.18035976962575426, 'nugget': 4.6861836446362183e-08}


dw_sim
Final Interpretable Params: {'sigma_sq': 11.817838113249758, 'range_lon': 0.22621206038334832, 'range_lat': 0.17422723320268027, 'range_time': 0.997943651842861, 'advec_lat': -0.050699303814975866, 'advec_lon': -0.18116598528943043, 'nugget': 1.1984570483507146}


vecc

Final Interpretable Params: {'sigma_sq': 13.719100200518179, 'range_lon': 0.32214277436468963, 'range_lat': 0.32239904147765264, 'range_time': 1.4623993761954481, 'advec_lat': -0.03314564565869727, 'advec_lon': -0.25639209965916504, 'nugget': 0.38030028721574305}


vecc_sim

Final Interpretable Params: {'sigma_sq': 12.967334679138654, 'range_lon': 0.1829644675670931, 'range_lat': 0.14094812906918186, 'range_time': 0.9219567362979819, 'advec_lat': -0.04561716644679645, 'advec_lon': -0.17524213405286115, 'nugget': 0.1315329566135699}






# TRUE PARAMETERS
init_sigmasq   = 13.059
init_range_lon = 0.195 
init_range_lat = 0.154 
init_advec_lat = 0.0418
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247


irregular grid

Final Interpretable Params: {
   'sigma_sq': 13.1581147614216, 
   'range_lon': 0.1670857937279293, 
   'range_lat': 0.13532624419142394, 
   'range_time': 0.662083817877393, 
   'advec_lat': 4.485362408040999, 
   'advec_lon': -0.05054465623325784, 
   'nugget': 3.202037263471046e-05}

regular grid

Final Interpretable Params: {
   'sigma_sq': 13.547646279613172, 
   'range_lon': 0.2063608511650159, 
   'range_lat': 0.1698378442441089, 
   'range_time': 1.122662499886297, 
   'advec_lat': 0.05415059834603563, 
   'advec_lon': -0.16640862287821778, 
   'nugget': 0.48795387825462566}




# fit dw

difference data

In [25]:
a = [11.0474, 0.0623, 0.2445, 1.0972, 0.0101, -0.1671, 1.1825]
day = 0 # 0 index
lat_range= [0,5]
lon_range= [123.0, 133.0]
#lat_range= [1,3]
#lon_range= [125, 129.0]

daily_aggregated_tensors_dw = [aggregated_data]
daily_hourly_maps_dw = [input_map]

db = debiased_whittle.debiased_whittle_preprocess(daily_aggregated_tensors_dw, daily_hourly_maps_dw, day_idx=day, params_list=a, lat_range=lat_range, lon_range=lon_range)


subsetted_aggregated_day = db.generate_spatially_filtered_days(0,5,123,133)
print(subsetted_aggregated_day.shape)
N2= subsetted_aggregated_day.shape[0]
print(N2)
subsetted_aggregated_day[:20]

torch.Size([142832, 4])
142832


tensor([[ 2.8000e-02,  1.2305e+02,  0.0000e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2311e+02,  1.9113e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2317e+02,  3.1846e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2323e+02,  9.3044e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2330e+02,  7.5384e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2336e+02, -8.2040e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2342e+02, -1.2852e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2349e+02, -3.2619e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2355e+02, -6.2629e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2361e+02,  1.3124e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2368e+02,  4.2001e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2374e+02, -4.3216e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2380e+02, -5.5636e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2386e+02, -2.0360e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2393e+02,  4.5638e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2399e+02, -5.2148e-02,  2.1000e+01],
        

In [26]:

dwl = debiased_whittle.debiased_whittle_likelihood()
if __name__ == '__main__':
    start_time = time.time()

    # --- Configuration ---
    DAY_TO_RUN = 3 # data is decided above
    TAPERING_FUNC = dwl.cgn_hamming # Use Hamming taper
    NUM_RUNS = 1
    MAX_STEPS = 20 # L-BFGS usually converges in far fewer steps
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {DEVICE}")

    DELTA_LAT, DELTA_LON = 0.044, 0.063 

    LAT_COL, LON_COL = 0, 1
    VAL_COL = 2 # Spatially differenced value
    TIME_COL = 3


    cur_df =subsetted_aggregated_day
    
    if cur_df.numel() == 0 or cur_df.shape[1] <= max(LAT_COL, LON_COL, VAL_COL, TIME_COL):
        print(f"Error: Data for Day {DAY_TO_RUN} is empty or invalid.")
        exit()

    unique_times = torch.unique(cur_df[:, TIME_COL])
    time_slices_list = [cur_df[cur_df[:, TIME_COL] == t_val] for t_val in unique_times]

    # --- 1. Pre-compute J-vector, Taper Grid, and Taper Autocorrelation ---
    print("Pre-computing J-vector (Hamming taper)...")
    
    # --- 💥 REVISED: Renamed 'p' to 'p_time' 💥 ---
    J_vec, n1, n2, p_time, taper_grid = dwl.generate_Jvector_tapered( 
        time_slices_list,
        tapering_func=TAPERING_FUNC, 
        lat_col=LAT_COL, lon_col=LON_COL, val_col=VAL_COL,
        device=DEVICE
    )

    if J_vec is None or J_vec.numel() == 0 or n1 == 0 or n2 == 0 or p_time == 0:
       print(f"Error: J-vector generation failed for Day {DAY_TO_RUN}.")
       exit()
       
    print("Pre-computing sample periodogram...")
    I_sample = dwl.calculate_sample_periodogram_vectorized(J_vec)

    print("Pre-computing Hamming taper autocorrelation...")
    taper_autocorr_grid = dwl.calculate_taper_autocorrelation_fft(taper_grid, n1, n2, DEVICE)

    if torch.isnan(I_sample).any() or torch.isinf(I_sample).any():
        print("Error: NaN/Inf in sample periodogram.")
        exit()
    if torch.isnan(taper_autocorr_grid).any() or torch.isinf(taper_autocorr_grid).any():
        print("Error: NaN/Inf in taper autocorrelation.")
        exit()

    print(f"Data grid: {n1}x{n2}, {p_time} time points. J-vector, Periodogram, Taper Autocorr on {DEVICE}.")
    # --- END REVISION ---

    # --- 2. Optimization Loop ---
    all_final_results = []
    all_final_losses = []

    for i in range(NUM_RUNS):
        print(f"\n{'='*30} Initialization Run {i+1}/{NUM_RUNS} {'='*30}")

        # --- 7-PARAMETER initialization ---
        ''' 
        init_sigmasq   = 15.0
        init_range_lat = 0.66 
        init_range_lon = 0.7 
        init_nugget    = 1.5
        init_beta      = 0.1  # Temporal range ratio
        init_advec_lat = 0.02
        init_advec_lon = -0.08
        '''
        init_sigmasq   = 13.059
        init_range_lat = 0.154 
        init_range_lon = 0.195
        init_advec_lat = 0.0218
        init_range_time = 0.7
        init_advec_lon = -0.1689
        init_nugget    = 0.247

        init_phi2 = 1.0 / init_range_lon
        init_phi1 = init_sigmasq * init_phi2
        init_phi3 = (init_range_lon / init_range_lat)**2
        # Change needed to match the spatial-temporal distance formula:
        init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

        initial_params_values = [
            np.log(init_phi1),    # [0] log_phi1
            np.log(init_phi2),    # [1] log_phi2
            np.log(init_phi3),    # [2] log_phi3
            np.log(init_phi4),    # [3] log_phi4
            init_advec_lat,       # [4] advec_lat (NOT log)
            init_advec_lon,       # [5] advec_lon (NOT log)
            np.log(init_nugget)   # [6] log_nugget
        ]
        
        print(f"Starting with FIXED params (raw log-scale): {[round(p, 4) for p in initial_params_values]}")

        params_list = [
            Parameter(torch.tensor([val], dtype=torch.float64))
            for val in initial_params_values
        ]

        # Helper to define the boundary globally for clarity
        NUGGET_LOWER_BOUND = 0.05
        LOG_NUGGET_LOWER_BOUND = np.log(NUGGET_LOWER_BOUND) # Approx -2.9957

        # --- 💥 REVISED: Use L-BFGS Optimizer 💥 ---
        optimizer = torch.optim.LBFGS(
            params_list,
            lr=1.0,           # Initial step length for line search
            max_iter=20,      # Iterations per step
            history_size=100,
            line_search_fn="strong_wolfe", # Often more robust
            tolerance_grad=1e-5
        )
        # --- END REVISION ---

        print(f"Starting optimization run {i+1} on device {DEVICE} (Hamming, 7-param ST kernel, L-BFGS)...")
        
        # --- 💥 REVISED: Call L-BFGS trainer, pass p_time 💥 ---
        nat_params_str, phi_params_str, raw_params_str, loss, steps_run = dwl.run_lbfgs_tapered(
            params_list=params_list,
            optimizer=optimizer,
            I_sample=I_sample,
            n1=n1, n2=n2, p_time=p_time,
            taper_autocorr_grid=taper_autocorr_grid, 
            max_steps=MAX_STEPS,
            device=DEVICE
        )
        # --- END REVISION ---
        
        if loss is not None:
            all_final_results.append((nat_params_str, phi_params_str, raw_params_str))
            all_final_losses.append(loss)
        else:
            all_final_losses.append(float('inf'))

    print(f"\n\n{'='*25} Overall Result from Run {'='*25} {'='*25}")
    
    valid_losses = [l for l in all_final_losses if l is not None and l != float('inf')]

    if not valid_losses:
        print(f"The run failed or resulted in an invalid loss for Day {DAY_TO_RUN}.")
    else:
        best_loss = min(valid_losses)
        best_run_index = all_final_losses.index(best_loss)
        best_results = all_final_results[best_run_index]
        
        print(f"Best Run Loss: {best_loss} (after {steps_run} steps)")
        print(f"Final Parameters (Natural Scale): {best_results[0]}")
        print(f"Final Parameters (Phi Scale)    : {best_results[1]}")
        print(f"Final Parameters (Raw Log Scale): {best_results[2]}")

    end_time = time.time()
    print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")

Using device: cpu
Pre-computing J-vector (Hamming taper)...
Pre-computing sample periodogram...
Pre-computing Hamming taper autocorrelation...
Data grid: 113x158, 8 time points. J-vector, Periodogram, Taper Autocorr on cpu.

============================== Initialization Run 1/1 ==============================
Starting with FIXED params (raw log-scale): [4.2042, 1.6348, 0.4721, -2.5562, 0.0218, -0.1689, -1.3984]
Starting optimization run 1 on device cpu (Hamming, 7-param ST kernel, L-BFGS)...
--- Step 1/20 ---
 Loss: 1.906929 | Max Grad: 6.274846e-04
  Params (Raw Log): log_phi1: 4.3182, log_phi2: 1.7745, log_phi3: 0.0776, log_phi4: -3.5323, advec_lat: 0.0363, advec_lon: -0.1510, log_nugget: -1.0186
--- Step 2/20 ---
 Loss: 1.817385 | Max Grad: 4.095367e-05
  Params (Raw Log): log_phi1: 4.3182, log_phi2: 1.7744, log_phi3: 0.0775, log_phi4: -3.5320, advec_lat: 0.0363, advec_lon: -0.1510, log_nugget: -1.0177
--- Step 3/20 ---
 Loss: 1.817385 | Max Grad: 4.095367e-05
  Params (Raw Log): log

init_sigmasq   = 13.059
init_range_lon = 0.195 
init_range_lat = 0.154 
init_advec_lat = 0.0418
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247

1 st simulation (1 vs 3)

Final Interpretable Params: {'sigma_sq': 12.939260896610579, 'range_lon': 0.17242543281507933, 'range_lat': 0.1631231346200311, 'range_time': 1.1358209227858689, 'advec_lat': 0.045119029109988974, 'advec_lon': -0.17809677784942035, 'nugget': 0.3009582276680578}

Final Parameters (Natural Scale): sigmasq: 13.2415, range_lat: 0.1685, range_lon: 0.1739, range_time: 0.9681, advec_lat: 0.0395, advec_lon: -0.1732, nugget: 0.3216

2nd simulation (1 vs 3)

Final Interpretable Params: {'sigma_sq': 12.765293287952144, 'range_lon': 0.17039368470743024, 'range_lat': 0.16152132799710625, 'range_time': 1.081124889959751, 'advec_lat': 0.04635511983762563, 'advec_lon': -0.1775715292039452, 'nugget': 0.31503884896742074}


sigmasq: 13.2415, range_lat: 0.1685, range_lon: 0.1739, range_time: 0.9681, advec_lat: 0.0395, advec_lon: -0.1732, nugget: 0.3216

3nd simulation (mm 15 two times)

Final Interpretable Params: {'sigma_sq': 12.292570063933866, 'range_lon': 0.16170186578939172, 'range_lat': 0.1537053192245325, 'range_time': 0.9671689003322674, 'advec_lat': 0.04155171168950867, 'advec_lon': -0.16218073366456207, 'nugget': 0.29945872751676345}

Final Parameters (Natural Scale): sigmasq: 12.7277, range_lat: 0.1631, range_lon: 0.1696, range_time: 0.9917, advec_lat: 0.0363, advec_lon: -0.1510, nugget: 0.3614
